# Test model response to reader input
Let's test whether the reader-aware model responds differently according to different reader group inputs.

In [1]:
# load test data
import torch
test_data = torch.load('../../data/reddit_data/combined_data_val_data.pt')
test_data_df = test_data.data.to_pandas()
print(len(test_data))
# add subreddit data lol
import pandas as pd
post_data = pd.read_csv('../../data/reddit_data/subreddit_submissions_2018-01_2019-12.gz', sep='\t', compression='gzip', usecols=['id', 'subreddit'])
post_data.rename(columns={'id' : 'article_id'}, inplace=True)
test_data_df = pd.merge(test_data_df, post_data, on='article_id', how='left')
display(test_data_df.head())

53527


,article_id,author_has_subreddit_embed,author_has_text_embed,reader_token,reader_token_str,source_text,source_text_reader_token,subreddit_embed,target_text,text_embed,source_ids,target_ids,attention_mask,source_ids_reader_token,subreddit
0,e7jfrg,False,False,6,UNK,6 months ago my wife had a prophylactic double...,6 months ago my wife had a prophylactic double...,"[1.2634457957357221, 1.1946322912923502, 2.131...",YTA Even if there was only one bottle in the b...,"[0.8712435804873842, -0.16627771240634429, 0.8...","[0, 401, 377, 536, 127, 1141, 56, 10, 1759, 37...","[0, 975, 3847, 1648, 114, 89, 21, 129, 65, 730...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 401, 377, 536, 127, 1141, 56, 10, 1759, 37...",AmItheAsshole
1,d7n9tt,False,False,6,UNK,We have a close friend's wedding coming up. My...,We have a close friend's wedding coming up. My...,"[0.4022668783185666, -0.9914351933707066, 0.41...","That being said, are you getting the glasses e...","[-0.38110951174974106, -1.023232965414158, -1....","[0, 170, 33, 10, 593, 1441, 18, 3312, 567, 62,...","[0, 1711, 145, 26, 6, 32, 47, 562, 5, 11121, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 170, 33, 10, 593, 1441, 18, 3312, 567, 62,...",AmItheAsshole
2,9z018r,False,False,6,UNK,"Hello, I am at a loss as to where to start her...","Hello, I am at a loss as to where to start her...","[-1.2149181563835465, 0.8477479133804404, 0.30...","Look at the cost of your debt for each, is the...","[-0.46578111210738665, -0.5391363140316221, 1....","[0, 31414, 6, 38, 524, 23, 10, 872, 25, 7, 147...","[0, 15833, 23, 5, 701, 9, 110, 1126, 13, 349, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 31414, 6, 38, 524, 23, 10, 872, 25, 7, 147...",personalfinance
3,8mly45,False,False,6,UNK,I’m thinking about joining the pc race but I c...,I’m thinking about joining the pc race but I c...,"[-1.3038122284781362, -0.09173205694986017, -0...",* What are you using it for?,"[-2.3937830229873303, -0.16802530920071082, 0....","[0, 100, 17, 27, 119, 2053, 59, 3736, 5, 46213...","[0, 3226, 653, 32, 47, 634, 24, 13, 116, 2, 1,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 100, 17, 27, 119, 2053, 59, 3736, 5, 46213...",pcmasterrace
4,coxtza,False,False,6,UNK,"I, 27M, am dating a long time friend, 26F. We ...","I, 27M, am dating a long time friend, 26F. We ...","[0.1198989489207567, -1.1244628791685227, -1.0...",Not to mention the fact that if she was expect...,"[1.2546455758042168, 0.973169048311534, -2.242...","[0, 100, 6, 974, 448, 6, 524, 4927, 10, 251, 8...","[0, 7199, 7, 4521, 5, 754, 14, 114, 79, 21, 48...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 100, 6, 974, 448, 6, 524, 4927, 10, 251, 8...",AmItheAsshole


In [35]:
## load model
from importlib import reload
import test_question_generation
reload(test_question_generation)
from test_question_generation import load_model
model_cache_dir = '../../data/model_cache/'
model_file = '../../data/reddit_data/author_text_data/question_generation_model/checkpoint-305500/pytorch_model.bin'
model_type = 'bart_author_token'
data_dir = '../../data/reddit_data/'
generation_model, model_tokenizer = load_model(model_cache_dir, model_file, model_type, data_dir)
generation_model.eval()
generation_model.to('cpu')

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50271, 768)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50271, 768)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): BartEncoderLayer(
       

## Sample data
Let's sample data that has at least one known response from one of the reader groups, and then compare the generated responses.

In [50]:
sample_data_j.head()

,article_id,author_has_subreddit_embed,author_has_text_embed,reader_token,reader_token_str,source_text,source_text_reader_token,subreddit_embed,target_text,text_embed,source_ids,target_ids,attention_mask,source_ids_reader_token,subreddit
39221,8u0705,False,False,4,<RESPONSE_TIME_0_AUTHOR>,My dad just gave me a life insurance policy th...,dad just gave me a life insurance policy they...,"[-0.7756150704752005, -0.22407624250187608, -2...",Does your job provide you with a life insuranc...,"[0.536935124880913, 1.0528294495793058, -1.084...","[0, 2387, 4252, 95, 851, 162, 10, 301, 1911, 7...","[0, 27847, 110, 633, 694, 47, 19, 10, 301, 191...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 4252, 95, 851, 162, 10, 301, 1911, 714, 51...",personalfinance
51175,ei2mth,False,False,5,<RESPONSE_TIME_1_AUTHOR>,"So, I usually overpay taxes on my paycheck. I ...",", I usually overpay taxes on my paycheck. I ha...","[-0.7841725882374827, -0.6671360849602653, -0....",Why are you having an extra 50 withheld every ...,"[0.03165991431565788, -0.5005817413079771, 0.4...","[0, 2847, 6, 38, 2333, 81, 14066, 2556, 15, 12...","[0, 7608, 32, 47, 519, 41, 1823, 654, 22292, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 6, 38, 2333, 81, 14066, 2556, 15, 127, 242...",personalfinance
38028,8zxwgg,False,False,5,<RESPONSE_TIME_1_AUTHOR>,"I'm not really great at math, but need some he...","'m not really great at math, but need some hel...","[-2.040786103639267, -0.5073124108991152, -0.3...","A few months of vacancy, a new roof here and t...","[-0.47193236484949913, -0.28554454313816674, 0...","[0, 100, 437, 45, 269, 372, 23, 10638, 6, 53, ...","[0, 250, 367, 377, 9, 20713, 6, 10, 92, 5645, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 437, 45, 269, 372, 23, 10638, 6, 53, 240, ...",personalfinance
46689,cg27uk,False,False,5,<RESPONSE_TIME_1_AUTHOR>,25/F who is currently $25K in debt. Approx. $9...,/F who is currently $25K in debt. Approx. $920...,"[-1.2752449028672168, -0.14977508455572314, -0...",How is your phone bill over $200?,"[-0.057852161966433314, 0.4384135281233764, 0....","[0, 1244, 73, 597, 54, 16, 855, 68, 1244, 530,...","[0, 6179, 16, 110, 1028, 1087, 81, 68, 2619, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 73, 597, 54, 16, 855, 68, 1244, 530, 11, 1...",personalfinance
37589,c3rpoq,False,False,5,<RESPONSE_TIME_1_AUTHOR>,Logged on to check balances this morning. Two ...,ged on to check balances this morning. Two acc...,"[-1.4353662817070942, 0.5577422081128921, -0.7...",What funds did you hold?,"[-1.6453263907199396, 0.03865495641902278, -0....","[0, 23345, 4462, 15, 7, 1649, 17172, 42, 662, ...","[0, 2264, 1188, 222, 47, 946, 116, 2, 1, 1, 1,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 4462, 15, 7, 1649, 17172, 42, 662, 4, 1596...",personalfinance


In [69]:
from model_helpers import generate_predictions
import numpy as np
from tqdm import tqdm
np.random.seed(123)
samples_per_group = 20
generation_params = {
    'generation_method' : 'sample',
    'temperature' : 1.0,
    'top_p' : 0.5,
}
max_source_length = len(test_data_df.loc[:, 'source_ids'].iloc[0])
max_target_length = len(test_data_df.loc[:, 'target_ids'].iloc[0])
length_penalty = 1
model_kwargs = []
author_group_category_lookup = {
    'location' : ['<US_AUTHOR>', '<NONUS_AUTHOR>'],
    'expert' : ['<EXPERT_PCT_0_AUTHOR>', '<EXPERT_PCT_1_AUTHOR>'],
    'response_time' : ['<RESPONSE_TIME_0_AUTHOR>', '<RESPONSE_TIME_1_AUTHOR>'],
}
pred_text_data = []
for subreddit_i, data_i in test_data_df.groupby('subreddit'):
    print(f'processing subreddit = {subreddit_i}')
    author_data_i = data_i[data_i.loc[:, 'reader_token_str']!='UNK']
    for author_category_j, author_groups in author_group_category_lookup.items():
        data_j = data_i[data_i.loc[:, 'reader_token_str'].isin(author_groups)]
        sample_data_j = data_j.loc[np.random.choice(data_j.index, min(samples_per_group, len(data_j)), replace=False), :]
        post_text_j = sample_data_j.loc[:, 'source_text']
        article_ids_j = sample_data_j.loc[:, 'article_id']
        # add each set of groups to text
        for author_group_k in author_groups:
            # add author token to text
            text_k = post_text_j + f' {author_group_k}'
            text_data_k = text_k.apply(lambda x: model_tokenizer.encode_plus(x))
            # fix data longer than max length
            text_data_k = text_data_k.apply(lambda x: {k : v[-max_target_length:] for k,v in x.items()})
            text_data_k = pd.DataFrame(text_data_k.values.tolist())
            input_id_k = text_data_k.loc[:, 'input_ids'].apply(lambda x: torch.LongTensor(x).unsqueeze(0))
            attention_k = text_data_k.loc[:, 'attention_mask'].apply(lambda x: torch.LongTensor(x).unsqueeze(0))
            output_k = []
            with torch.no_grad():
                for input_id_l, attention_l in tqdm(zip(input_id_k, attention_k)):
                    output_l = generation_model.generate(
                        input_ids=input_id_l,
                        attention_mask=attention_l,
                        top_p=generation_params['top_p'],
                        temperature=generation_params['temperature'],
                        max_length=max_target_length,
                        length_penalty=length_penalty,
                        num_return_sequences=1,
                        do_sample=True,
                    )
                    output_k.append(output_l[0])
            pred_text_k = [model_tokenizer.decode(ids, skip_special_tokens=True) for ids in output_k]
            pred_data_df_k = pd.DataFrame(pred_text_k, columns=['question'])
            pred_data_df_k = pred_data_df_k.assign(**{
                'post' : post_text_j.values,
                'author_group_category' : author_category_j,
                'author_group' : author_group_k,
                'subreddit' : subreddit_i,
                'id' : article_ids_j.values,
            })
            pred_text_data.append(pred_data_df_k)
pred_text_data = pd.concat(pred_text_data, axis=0)

0it [00:00, ?it/s]

processing subreddit = Advice


20it [00:13,  1.53it/s]
20it [00:14,  1.39it/s]
20it [00:11,  1.73it/s]
20it [00:11,  1.76it/s]
20it [00:13,  1.50it/s]
20it [00:11,  1.67it/s]
0it [00:00, ?it/s]

processing subreddit = AmItheAsshole


20it [00:12,  1.57it/s]
20it [00:15,  1.26it/s]
20it [00:16,  1.23it/s]
20it [00:15,  1.30it/s]
20it [00:14,  1.42it/s]
20it [00:14,  1.40it/s]
0it [00:00, ?it/s]

...

20it [00:14,  1.39it/s]
20it [00:14,  1.42it/s]
20it [00:14,  1.42it/s]
20it [00:13,  1.43it/s]
20it [00:13,  1.51it/s]
20it [00:15,  1.32it/s]
0it [00:00, ?it/s]

processing subreddit = pcmasterrace


20it [00:14,  1.43it/s]
20it [00:12,  1.62it/s]
20it [00:13,  1.53it/s]
20it [00:12,  1.56it/s]
20it [00:14,  1.37it/s]
20it [00:12,  1.55it/s]
0it [00:00, ?it/s]

processing subreddit = personalfinance


20it [00:11,  1.67it/s]
20it [00:11,  1.69it/s]
20it [00:14,  1.40it/s]
20it [00:15,  1.33it/s]
20it [00:14,  1.41it/s]
20it [00:16,  1.22it/s]


In [70]:
display(pred_text_data.head())

,question,post,author_group_category,author_group,subreddit,id
0,"To date, how have you been able to date?",What steps should I take for this? I am not in...,location,<US_AUTHOR>,Advice,c6nfm0
1,"Also, how do you lose interest in the subject?",I'm in my second year in college majoring in m...,location,<US_AUTHOR>,Advice,dhrso4
2,"Well, how old are you?",Okay if a guy makes time every week to fw you ...,location,<US_AUTHOR>,Advice,e567z8
3,How did you achieve them?,Hi.I feel very lonely.I made my 1rst friend in...,location,<US_AUTHOR>,Advice,dj4mx1
4,How long has this been going on?,My dog (8 month husky) went missing last week ...,location,<US_AUTHOR>,Advice,ci2iqc


Let's look for questions that different between different people in the same category.

In [77]:
from itertools import combinations
# post | category | group_1 | question_1 | group_2 | question_2
differ_question_data = []
for (category_i, article_id_i), data_i in pred_text_data.groupby(['author_group_category', 'id']):
    post_text_i = data_i.loc[:, 'post'].iloc[0]
    subreddit_i = data_i.loc[:, 'subreddit'].iloc[0]
    # get author group combos
    group_category_combos_i = combinations(data_i.loc[:, 'author_group'].unique(), r=2)
    for author_group_1, author_group_2 in group_category_combos_i:
        data_i_1 = data_i[data_i.loc[:, 'author_group']==author_group_1]
        data_i_2 = data_i[data_i.loc[:, 'author_group']==author_group_2]
        for idx_j, data_j_1 in data_i_1.iterrows():
            question_1 = data_j_1.loc['question']
            for idx_k, data_k_2 in data_i_2.iterrows():
                question_2 = data_k_2.loc['question']
                if(question_1 != question_2):
                    differ_question_data_k = pd.Series([
                        category_i, article_id_i, post_text_i, subreddit_i, question_1, question_2, author_group_1, author_group_2,
                    ], index=['author_group_category', 'id', 'post', 'subreddit', 'question_1', 'question_2', 'author_group_1', 'author_group_2'])
                    differ_question_data.append(differ_question_data_k)
differ_question_data = pd.concat(differ_question_data, axis=1).transpose()

In [82]:
pd.set_option('display.max_colwidth', 500)
for (subreddit_i, category_i), data_i in differ_question_data.groupby(['subreddit', 'author_group_category']):
    print(f'**** differing questions for author category={category_i}; subreddit={subreddit_i} ****')
    display(data_i.head(5))

**** differing questions for author category=expert; subreddit=Advice ****


,author_group_category,id,post,subreddit,question_1,question_2,author_group_1,author_group_2
5,expert,8uwk5j,"I (34F) would really like a child and I have a friend (40M) who also would. Both of us are having trouble finding people in the romance department and our clocks are ticking. The subject has been broached many times over the years, both of I think joking and half serious, but now that our ages are getting up there, I think it's a more serious discussion. My question is what kind of questions do we ask each other to see if we're a right fit to parent together?",Advice,How serious are you guys?,How serious is the relationship?,<EXPERT_PCT_0_AUTHOR>,<EXPERT_PCT_1_AUTHOR>
6,expert,90iiy9,"I get angry/jealous when I see other people doing cool things. How can I work on this? My life is pretty good: I do cook things, have nice stuff, etc. But I always look at other people who are doing things I think are cool and get annoyed at them rather than happy for them. Can you help me stop?",Advice,How would you describe your life?,How would you describe your ideal life?,<EXPERT_PCT_0_AUTHOR>,<EXPERT_PCT_1_AUTHOR>
7,expert,95w213,"My friend, who has a stunning body and perfect thighs will sometimes bring up how thin mine are. I often try new exercise plans to thicken them out but I find it hard to stick to them as they often do not affect me or take too much time out of my crammed day. Is there any programs that I could maybe fit into a busy schedule and would have an effect on the shape of my legs? Thank you.",Advice,How many different lies do you have on your tongue?,How many lies do you have on your lies?,<EXPERT_PCT_0_AUTHOR>,<EXPERT_PCT_1_AUTHOR>
12,expert,9ee0gs,"Sorry if the post is a little long but I want to make sure all the facts are there. In April my car was suffering from leaks in two of its fuel lines. I had gotten estimates from $1,250 to $2,500 from multiple shops in my area and being a broke college student I can’t afford that and even if I could Id rather put that much money into a new car than an older one that isn’t even worth that much money. I told my parents what was going on and that I was preparing to buy a new car but my dad told...",Advice,Was he ever willing to pay for it?,What's the cost of the car?,<EXPERT_PCT_0_AUTHOR>,<EXPERT_PCT_1_AUTHOR>
14,expert,9iimhg,My first credit card has a $500 limit and never gets used anymore. It’s connected to my account at the bank where I have a checking and savings account. I’m thinking about switching everything to a different bank but don’t want to leave the credit card behind. Can I cancel it without a problem?,Advice,Or is the card literally only used if you have a checking account?,How the hell did you find out about the new bank?,<EXPERT_PCT_0_AUTHOR>,<EXPERT_PCT_1_AUTHOR>


**** differing questions for author category=location; subreddit=Advice ****


,author_group_category,id,post,subreddit,question_1,question_2,author_group_1,author_group_2
84,location,9z0a8c,"This will be difficult to summarize, and is going to be missing some details, either due to my own fault, or to save time for the reader. I don't mind elaborating on specifics, if need be.The short version of my problem is: I live in the bay area and I have an ailing father of whom I have to take care of, a 7-month old son, wife, and I'm not making enough money to take care of all of them. But, there's more to all of this, of which I will still try to summarize in their own categories below....",Advice,Was the money withheld from your paycheck?,Was the money withheld for child support or child support?,<US_AUTHOR>,<NONUS_AUTHOR>
86,location,aceyyz,All I wanted was to marry her and make her happy. Now it’s over.What next?,Advice,Did you ever talk about her happiness?,"First of all, how long have you been together?",<US_AUTHOR>,<NONUS_AUTHOR>
95,location,beaavq,"Disclaimer: the text got a bit long, I am sorry.Throw-away, because a friend of mine recently figured out my Reddit name. And sorry for any grammar fauxpas, english is not my mother tongue. First things first I recently got 20 years old and a big (really big) hobby of mine is playing video games, mostly competitive shooter on PC.I built my first PC about two and a half years ago(before I only played cod on PlayStation).I quickly got into csgo, later into pubg, Overwatch, apex and co.At first...",Advice,"Well, how do you get to the point where you don't know what to do or don't do?","Well, how do you get the idea of gaming?",<US_AUTHOR>,<NONUS_AUTHOR>
99,location,c57k61,"Basically, one of my uncles (lets call him A) is coming to my city for the night. He messaged me and my other uncle (lets call him B) asking if either of us were free and wanted to meet up. This is my only night off work and I messaged A telling him I'd love to see him. B then replied inviting A as a plus one to some museum exhibition thing, adding that if A couldn't go then I was invited to the museum thing.The problem is I really don't want to go the the museum thing, it sounds boring and ...",Advice,Did you already have an activity last week?,Check out any local businesses you already have dinner with and see what they do?,<US_AUTHOR>,<NONUS_AUTHOR>
100,location,c6nfm0,"What steps should I take for this? I am not interested in actually learning how to speak, I am just big into books and want to read some Japanese books",Advice,"To date, how have you been able to date?","To date, how have you been with the books?",<US_AUTHOR>,<NONUS_AUTHOR>


**** differing questions for author category=response_time; subreddit=Advice ****


...

,author_group_category,id,post,subreddit,question_1,question_2,author_group_1,author_group_2
0,expert,8kcnq2,"I feel that not enough people know how great Costco is. Besides cheap toilet paper, organic produce, and hot dogs, they also have an auto program for members. Simply call the auto program phone number, give them your membership number, tell them the car you want, and they will have a dealership contact you. You will often receive under-invoice pricing for popular models and sometimes extra goodies thrown in. I personally know people who have gotten great deals on cars like a new Toyota Corol...",personalfinance,Is it for a good price?,Is it for a $60 or for a year?,<EXPERT_PCT_0_AUTHOR>,<EXPERT_PCT_1_AUTHOR>
2,expert,8rpivh,"Hi, I am an 18 year old and a recent high school graduate. I was admitted to a high-tier college with the price tag similar to that of an ivy league school. I recently got into a disagreement with my parents, and now they refuse to pay for my college (or living costs). Is there a way for me to finance my college education (the payment for the first term, roughly $30K if you include housing and meals, is due before September)?I come from a relatively wealthy family, and thus I do not qualify ...",personalfinance,How is the work expected to pay for it?,How is your work expected to pay for it?,<EXPERT_PCT_0_AUTHOR>,<EXPERT_PCT_1_AUTHOR>
9,expert,99ujfo,"I’m not sure if this is the right place to post this so, I apologise. Most of my tuition is covered by financial aid. However I still need to pay 10,000 per term. Which is thrice a year. So far I’ve been doing online jobs and such holding onto the fact that soon I’d get my work permit and I wouldn’t have to keep going that. But. I just found out that the work permit is only to fulfil a 40 hour learning requirement, restricted to my major. How can I earn the money if I’m not legally allowed ...",personalfinance,Did you fill out a W-4 or a W9 when you found out about the work permit?,"Also, how old are you?",<EXPERT_PCT_0_AUTHOR>,<EXPERT_PCT_1_AUTHOR>
15,expert,9k1b1b,"Alright so first thing's first.Net Income: $3200 a month. ($1600 bi-weekly $26.50/hr at current job)Extra Income: $1400 (GI bill BAH)Amount in bank: $11,500Monthly expenses:* Rent: $650 (I share a home with two people. Rate includes utils/internet/etc.)* Car Insurance: $100 (It's paid every six months but I count the amount I pay per month over that period)* Gasoline: $100* Groceries: $300 (This includes non-edible consumables TP,Toothpaste etc)* Meds: $8 (VA healthcare)* Car Payment: $150 (...",personalfinance,How many meals do you have in your budget?,How many days a week do you work?,<EXPERT_PCT_0_AUTHOR>,<EXPERT_PCT_1_AUTHOR>
16,expert,9kvk3x,"Hey all. Feeling a bit down with my current job situation currently. It seems like no matter where I apply to, I dont get the job. No matter where it is.I've applied to retail stores, fast food restaurants, customer service, warehouse jobs. It all ends the same, with nothing happening. Or getting an email stating I dont meet the qualifications for said job.I feel as if i have marketable skills:Military ExperienceLeadershipOffice experience.Customer service Manual labor.88 credit hours total ...",personalfinance,How many hours did you work on the job?,How many hours did you work in the job?,<EXPERT_PCT_0_AUTHOR>,<EXPERT_PCT_1_AUTHOR>


**** differing questions for author category=location; subreddit=personalfinance ****


,author_group_category,id,post,subreddit,question_1,question_2,author_group_1,author_group_2
67,location,7wkm90,"I'm living overseas for the inevitable future, but I want to maintain my US credit score in case I move backMy current card in the basic Bank of America one, so it's overall shitty (have to spend 10$ minimum each month, charged international fees, bad rate)The ideal credit card would be one that doesn't have a monthly minimum or annual fee (fat chance). I am also open to:1. A card with a low monthly minimum with no international fees2. A card with a low annual fee and no monthly minimumDoes...",personalfinance,Does the card have an annual fee?,Do you have any credit history whatsoever?,<US_AUTHOR>,<NONUS_AUTHOR>
71,location,896jt4,"General question: Does it ever make sense to sell your house because its at an all time high and rent for a year or two? If so how is the best way to calculate if its right decision for you? Specific to me: For this exercise lets take all human elements way such as family, job change, life events etc. I lived in the same condo since 2007 (refinanced in 2012). In its current amortization I pay $370 /mo towards principle and $600 to interest ($970 mortgage). The interest rate on this...",personalfinance,How much do you owe on the condo?,How much is the property worth?,<US_AUTHOR>,<NONUS_AUTHOR>
75,location,8k0dt0,I have two higher limit cards that I use each month (swap every other month) and pay off. Is it work keeping these other two open or close them out? Average credit here.,personalfinance,"What are your financial goals for the next year, 5 years, 10 years, 30 years?",Is there a reason you need more than two cards?,<US_AUTHOR>,<NONUS_AUTHOR>
79,location,97jk5o,"I’m living south western Ontario where housing is cheaper than say Toronto. My parents have a house next to where I live which they own and no mortgage attached. They want to sell it to me for 160,000 which is much lower than anything around as a house down the street sold for 329,000 which was essentially the same. The house needs renovations maybe 30-50k. The house is in an okay area I wouldn’t say it’s the greatest. The expect me to live there for minimum 10 years and that’s their only r...",personalfinance,How do you know that they're not still living in your house?,"Also, where do you live that the mortgage is 320k?",<US_AUTHOR>,<NONUS_AUTHOR>
85,location,a23lb2,"I've had a whole life policy for over 30 years now. It was started by my parents when I was an infant, and I took over paying the premiums about 5 years ago.The death benefit is $125k and current surrender is right at $4k. My premiums are $125/quarter. I think the guaranteed interest rate is 4.5%, but I honestly don't even really look at that since I don't really think about the money as any type of investment (Because it's not...).I'm 35, married with 0 kids right now and do have a term pol...",personalfinance,How many months of expenses does the policy cover?,How many years of your life?,<US_AUTHOR>,<NONUS_AUTHOR>


**** differing questions for author category=response_time; subreddit=personalfinance ****


,author_group_category,id,post,subreddit,question_1,question_2,author_group_1,author_group_2
130,response_time,8p1z07,"Hello everyone, I hope this is an appropriate place to pose this question, and I apologize in advance if it is not. About 3 weeks ago I got food poisoning from the restaurant I work at, (which 2 managers confirmed was due to the kitchen not following proper procedures), I vomited over 100 times and was unconscious basically for around 10 hours. The ER ran a bunch of tests, gave me a ton of medicine and fluids. I get a bill today saying I owe $1700 after my insurance covered the rest. And the...",personalfinance,Did you provide the insurance information to them?,What's the statute of limitations on insurance?,<RESPONSE_TIME_0_AUTHOR>,<RESPONSE_TIME_1_AUTHOR>
132,response_time,8u0705,"My dad just gave me a life insurance policy they took out on me when I was born for 25k (plus paid-up additions), and told me now that I'm married, it's my responsibility. Only $116 bucks a year, so I'll probably stick with it, but now I'm just thinking about everything else I need to do. Should I get an additional policy on top of this?Also, as a side question, we signed a pre-nup due to my family business (shares in the company, etc etc) but what were to happen if I died? Would she be home...",personalfinance,Was she living there before you were married?,Was her name listed as the owner of the house?,<RESPONSE_TIME_0_AUTHOR>,<RESPONSE_TIME_1_AUTHOR>
134,response_time,8uy1um,So a couple of months ago I was looking at chase credit journey and my credit score went from 747 to 658 in a month and it has been just floating there. There is absolutely no reason for it to drop that much and hover there. My fico credit score is still great but no one is able to explain to me why my credit dropped. I called trans union they say they can't tell me the information as it is their policy. I asked them to send my their policy they refused. I called chase and they said to talk ...,personalfinance,What’s your location?,What is the reason they refused to talk to you?,<RESPONSE_TIME_0_AUTHOR>,<RESPONSE_TIME_1_AUTHOR>
135,response_time,8vuwxc,"Hope this is allowed - posted this in r/insurance and they suggested adding more insurance. My brother believes I have 'too much' insurance. Welcome other perspectives:Located: Arlington, VA (near Washington, DC)Current Insurance: Erie. Will look at USAA, AAA, Geico, All State, Travelers, and State Farm.I'm currently in the process of reviewing my Auto Insurance Coverage and wanted to ensure I had enough. I think I might have too much.I Drive: less than 7K/annually. I telework and only dri...",personalfinance,How much did you pay the car and how much was the value of the car?,Did you take the car to the dealer and have it inspected by a mechanic before you bought it?,<RESPONSE_TIME_0_AUTHOR>,<RESPONSE_TIME_1_AUTHOR>
136,response_time,8xfe3q,"My \[25F\] FH \[35M\] and I are paying almost 4200 a year for full coverage car ins. on a 2014 Kia Soul, and a 2015 GMC Terrain through progressive. I have one speeding ticket from 2015 and he has one fender bender from 2014, but that is it. It just seems ridiculously high for car insurance especially in Missouri! I am hopeful that our ins. will go down in November after we get married though I am not holding my breath, it didn’t go down when I was 25 and it kept going UP before when I had z...",personalfinance,How high did you go for?,How high is the alcohol?,<RESPONSE_TIME_0_AUTHOR>,<RESPONSE_TIME_1_AUTHOR>


Paraphrase questions:

- `Advice`, `expert`: `How would you describe your life?` (non-expert) vs. `How would you describe your ideal life?` (expert)
- `AmItheAsshole`, `time`: `How did you respond?` (slow) vs. `Also, how did you respond to her?` (fast)
- `legaladvice`, `location`: `Have you asked the company where the check was?` (US) vs. `Have you tried asking them?` (non-US)
- `pcmasterrace`, `expert`: `Also, what power supply do you have?` (non-expert) vs. `Also what power supply do you have?` (expert)
- `pcmasterrace`, `location`: `What parts do you have?` (US) vs. `What parts do you have exactly?` (non-US)
- `personalfinance`, `time`: `Was she living there before you were married?` (fast) vs. `Was her name listed as the owner of the house?` (slow)

Questions with differing information goals:
- `Advice`, `location`: `Did you ever talk about her happiness?` (US) vs. `First of all, how long have you been together?` (non-US)
- `Advice`, `time`: `What's going on?` (fast) vs. `How old are you?` (slow)
- `legaladvice`, `expert`: `Also, did the cops come out when the domestic situation happened?` (non-expert) vs. `What did the cops say when they came out?` (expert)
- `pcmasterrace`, `expert`: `Have you tried reinstalling the software, turning it off and on again?` (non-expert) vs. `What settings are you using?` (expert)
- `pcmasterrace`, `time`: `What do you have for a CPU and GPU currently?` (fast) vs. `which monitor do you have?` (slow)
- `personalfinance`, `expert`: `How many meals do you have in your budget?` (non-expert) vs. `How many days a week do you work?` (expert)
- `personalfinance`, `location`: `What are your financial goals for the next year, 5 years, 10 years, 30 years?` (US) vs. `Is there a reason you need more than two cards?` (non-US)
- 


### Predict reader group of generated text
Let's use some of the classification models that we've trained to identify questions that appear to be written by a particular reader group.

Ideally, we will find that text generated by the reader-aware models will score higher than text generated by the reader-agnostic model.

In [3]:
## load model
## get model, tokenizer
from importlib import reload
import test_author_group_prediction
reload(test_author_group_prediction)
from test_author_group_prediction import load_model_tokenizer
model_name = 'facebook/bart-base'
model_cache_dir = '../../data/model_cache/'
num_labels = 2
# get model weights
import os
group_var = 'expert_pct_bin'
model_dir = f'../../data/reddit_data/group_classification_model/group={group_var}/'
checkpoint_dir = max(list(filter(lambda x: x.startswith('checkpoint'), os.listdir(model_dir))), key=lambda x: int(x.split('-')[1]))
checkpoint_dir = os.path.join(model_dir, checkpoint_dir)
model_weight_file = os.path.join(checkpoint_dir, 'pytorch_model.bin')
model, tokenizer = load_model_tokenizer(model_name, model_cache_dir, num_labels=num_labels, model_weight_file=model_weight_file)

Some weights of the model checkpoint at facebook/bart-base were not used when initializing BartForSequenceClassification: ['final_logits_bias']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.out_proj.bias', 'classification_head.out_proj.weight', 'classification_head.dense.bias', 'classification_head.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to u

In [8]:
## load test data
import torch
from test_author_group_prediction import BasicDataset
test_data = BasicDataset()
test_data = torch.load(os.path.join(model_dir, 'test_data.pt'))

AttributeError: Can't get attribute 'BasicDataset' on <module '__main__'>